# 01 – EDA and Baseline Setup

## 1. Imports and paths

## 2. Load raw data

## 3. Basic sanity checks

## 4. Create target (home_win)

## 5. Quick baseline model using moneyline

## 6. Save processed table